In [1]:
import pandas as pd
import numpy as np
import json
import math

In [47]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV, train_test_split

In [89]:
a = ["horario","atrito"]
b = [
    {"id_assunto":170,"horario":"sim","atrito":"sim"},
    {"id_assunto":160,"horario":"sim","atrito":"não"},
    {"id_assunto":170,"horario":"não","atrito":"não"}
]
c = ["id_assunto","horario"]

In [90]:
teste = pd.DataFrame(b)

In [91]:
teste

,id_assunto,horario,atrito
0,170,sim,sim
1,160,sim,não
2,170,não,não


In [92]:
features = teste[c]

In [93]:
features

,id_assunto,horario
0,170,sim
1,160,sim
2,170,não


In [61]:
with open("Datassets/myjsonfile.json",'r',encoding='UTF-8') as file:
    datasset = json.load(file)

In [88]:
pd.DataFrame(datasset['data'])[c]

,id_assunto
0,57
1,82
2,170
3,176
4,57
...,...
43995,244
43996,244
43997,258
43998,258


In [96]:
class Modeling:
    def __init__(self,datasset,targets) -> None: #features
        
        self.dataframe = pd.DataFrame(datasset)
        self.target = self.dataframe[targets]
        self.features = self.dataframe.drop(targets,axis=1)

        self.uniqueTargetsCount= {i:{"UniqueValues":np.bincount(self.dataframe[i]),"UniqueProbability":np.bincount(self.dataframe[i])/(len(self.dataframe[i])),"informationGain":{}} for i in targets}
    
        for k in self.uniqueTargetsCount:

            for j in self.features:

                self.uniqueTargetsCount[k]["informationGain"][j] = self.information_gain(self.dataframe,j,k,self.uniqueTargetsCount[k]["UniqueProbability"])

        self.encoded_features = self.label_encoder(self.features)
        self.X_train, self.X_test, self.y_train, self.y_test = self.train_test(self.encoded_features,self.target)
    
    def calc_entropy(self,column,UniqueProbability):
        # Conta o número de ocorrências de cada valor único na coluna
        #UniqueProbability
        
        # Calcula a probabilidade de cada valor dividindo pela quantidade total de elementos na coluna
        probabilities = UniqueProbability / len(column)
        
        entropy = 0  # Valor inicial da entropia
        
        # Loop para calcular a entropia de cada valor único
        for prob in probabilities:
            if prob > 0:
                entropy += prob * math.log(prob, 2)  # Calcula a entropia de cada valor e soma ao total

        return -entropy  # Retorna o negativo da entropia conforme a fórmula
    
    def information_gain(self,data, split,target,UniqueProbability):

        original_entropy=self.calc_entropy(data[target],UniqueProbability)

        values=data[split].unique()

        if len(values) < 2:
            # Se houver menos de dois valores únicos, não é possível dividir, então o ganho de informação é 0
            return 0

        left_split=data[data[split]==values[0]]

        right_split=data[data[split]==values[1]]

        subract=0

        for subset in [left_split,right_split]:
            
            prob=(subset.shape[0])/data.shape[0]
            subract += prob * self.calc_entropy(subset[target],UniqueProbability)

        return  original_entropy - subract
    
    def label_encoder(self,X):
        Encoder_X=LabelEncoder()

        for col in X.columns:
            X[col]=Encoder_X.fit_transform(X[col])
        
        return X
    
    def train_test(self,X,y):

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        return X_train, X_test, y_train, y_test


In [97]:
myModeling = Modeling(datasset['data'],['id_assunto'])

In [107]:
myModeling.X_test.describe()

,mensagem_resposta,data_hora_analise,data_hora_encaminhado,data_hora_assumido,data_hora_execucao,id_contrato_kit,preview,data_agenda_final,id,tipo,...,data_prazo_limite,data_reservada,id_ticket,origem_endereco,justificativa_sla_atrasado,origem_endereco_estrutura,data_reagendar,data_prev_final,origem_cadastro,ultima_atualizacao
count,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.0,8800.000000,8800.000000,8800.000000,...,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000,8800.000000
mean,9025.557500,36.486477,4092.811818,0.191705,2067.823864,7112.439432,0.0,84.974773,22122.243182,1.106136,...,30.306932,38.098977,7258.123182,2.423295,5.826818,1.188864,52.731477,0.472386,0.971136,20069.729432
std,7121.661644,195.499172,5565.478013,0.539558,3777.618935,5009.555022,0.0,368.128547,12721.013848,0.308767,...,106.900978,57.070819,6504.768053,1.033600,46.887305,0.455811,271.212512,0.499265,0.167433,11627.251836
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000
25%,1860.500000,0.000000,0.000000,0.000000,0.000000,2681.750000,0.0,0.000000,11184.750000,1.000000,...,0.000000,1.000000,1.000000,2.000000,0.000000,1.000000,0.000000,0.000000,1.000000,9829.750000
50%,7498.000000,0.000000,1.000000,0.000000,0.000000,7054.000000,0.0,1.000000,22199.000000,1.000000,...,1.000000,1.000000,6355.000000,2.000000,0.000000,1.000000,0.000000,0.000000,1.000000,20046.500000
75%,15640.750000,0.000000,8209.500000,0.000000,2532.000000,11358.250000,0.0,1.000000,33201.000000,1.000000,...,1.000000,77.000000,13135.500000,3.000000,0.000000,1.000000,1.000000,1.000000,1.000000,30177.250000
max,21798.000000,1604.000000,17318.000000,24.000000,13463.000000,16209.000000,0.0,2440.000000,43997.000000,2.000000,...,653.000000,194.000000,19554.000000,4.000000,571.000000,2.000000,2131.000000,1.000000,1.000000,40146.000000
